<a href="https://colab.research.google.com/github/foksly/multiple-choice-QA/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-pretrained-bert

    100% |████████████████████████████████| 122kB 3.8MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls "/content/drive/My Drive/Colab Notebooks/Diplom"

allNoYear.csv		      fasttext_embs.pkl		siamese.py
allNoYearFastText.pickle      json_test.ipynb		supervised.py
BERT.ipynb		      model.pt			swag.csv
Diplom.ipynb		      multiple-choice-QA.ipynb	unsupervised.py
elmo_embs_isYearFalse.pickle  myutils.py		utils.py
elmo_embs.pkl		      __pycache__


In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks/Diplom/')

import utils, unsupervised, supervised, siamese

In [0]:
import json
import pandas as pd
import numpy as np

from gensim.models.wrappers import FastText
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import re

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Diplom/allNoYear.csv")

In [0]:
prog = re.compile('[0-4]+')
def clear_answer(s):
  return int(prog.findall(s)[0])

In [0]:
df['ca'] = df['ca'].apply(lambda x: clear_answer(x))

In [0]:
class InputFeatures(object):
    def __init__(self,
                 example_id,
                 choices_features,
                 label

                ):
        self.example_id = example_id
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label

In [0]:
def prepare_for_bert(df, tokenizer, max_seq_len=265):
  features = []
  for i in tqdm(range(len(df))):
    choices_features = []
    
    line = df.iloc[i]
    question = ['[CLS]'] + tokenizer.tokenize(line['q']) + ['[SEP]']
    answers = [line['a1'], line['a2'], line['a3'], line['a4']]
    for a in answers:
      choice = tokenizer.tokenize(a) + ['[SEP]']
      tokens = question + choice
      segment_ids = ([0] * len(question)) + ([1] * len(choice))
      
      input_ids = tokenizer.convert_tokens_to_ids(tokens)
      input_mask = [1] * len(input_ids)
      
      # padding
      padding = [0] * (max_seq_len + 3 - len(input_ids))
      input_ids += padding
      input_mask += padding
      segment_ids += padding
      
      choices_features.append((tokens, input_ids, input_mask, segment_ids))
    
    label = line['ca']
    
    features.append(
            InputFeatures(
                example_id = i,
                choices_features = choices_features,
                label = label
            )
        )

  return features
    
    

In [11]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt not found in cache, downloading to /tmp/tmp5lf56s3m


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


100%|██████████| 995526/995526 [00:00<00:00, 12202327.01B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmp5lf56s3m to cache at /root/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmp5lf56s3m
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [0]:
from __future__ import absolute_import

import argparse
import csv
import logging
import os
import random
import sys
from io import open

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import (BertForMultipleChoice, BertConfig, WEIGHTS_NAME, CONFIG_NAME)
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear
from pytorch_pretrained_bert.tokenization import BertTokenizer

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
features = prepare_for_bert(df, tokenizer)

In [0]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

def select_field(features, field):
    return [
        [
            choice[field]
            for choice in feature.choices_features
        ]
        for feature in features
    ]

In [0]:
model = BertForMultipleChoice.from_pretrained('bert-base-multilingual-cased', 
                                              cache_dir=os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE)), 
                                              num_choices=4)
model.to(device)

In [0]:
param_optimizer = list(model.named_parameters())

In [0]:
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

In [0]:
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

In [0]:
num_train_epochs = 5
num_train_optimization_steps = int(len(df) / 32) * num_train_epochs

In [0]:
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=5e-5,
                     warmup=0.1,
                     t_total=num_train_optimization_steps)

In [0]:
train_features = prepare_for_bert(df, tokenizer)

In [34]:
global_step = 0
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(df))
logger.info("  Batch size = %d", 32)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor(select_field(train_features, 'input_ids'), dtype=torch.long)
all_input_mask = torch.tensor(select_field(train_features, 'input_mask'), dtype=torch.long)
all_segment_ids = torch.tensor(select_field(train_features, 'segment_ids'), dtype=torch.long)
all_label = torch.tensor([f.label for f in train_features], dtype=torch.long)
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

INFO:__main__:***** Running training *****
INFO:__main__:  Num examples = 3060
INFO:__main__:  Batch size = 32
INFO:__main__:  Num steps = 475


In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.train()
for _ in trange(int(num_train_epochs), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        loss = model(input_ids, segment_ids, input_mask, label_ids)
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1

Iteration:   0%|          | 0/96 [00:00<?, ?it/s]



RuntimeError: ignored

In [0]:
text = '[CLS] Совещательный орган при императоре в начале XX в. [SEP] Сенат [SEP]'
tokenized_text = tokenizer.tokenize(text)

In [35]:
tokenized_text

['[CLS]',
 'сов',
 '##е',
 '##ща',
 '##тель',
 '##ны',
 '##и',
 'орган',
 'при',
 'император',
 '##е',
 'в',
 'начале',
 'xx',
 'в',
 '.',
 '[SEP]',
 'се',
 '##нат',
 '[SEP]']